In [1]:
file_path = "C:/Azure_kinect_project/test_MS1/test_MS2/reference1.txt"

f = open(file_path, 'r')
lines = f.readlines()

output = []
frame = []
joint = 0
for line in lines:
    joint+=1
    line = line.strip()  # 줄 끝의 줄 바꿈 문자를 제거한다.
    line = line.split(":")
    line = line[1]
    
    line = line.strip(')')
    line = line.strip('(')
    line = line.split(",")

    for i in range(3):
        line[i] = float(line[i])
    frame.append(line)
    
    if joint%32 == 0:
        output.append(frame)
        joint = 0
        frame = []
f.close() 
#print(output)

In [2]:
import numpy as np

In [3]:
### 최대각도 구하기 #1 ###
#output[frame][joint][x,y,z]

# ∠
# the joints in test limb
angleList = []
for i in range (len(output)):
    joint_1_xyz = np.array(output[i][4])  # a
    joint_2_xyz = np.array(output[i][7])  # b
    # the joints in plane
    joint_3_xyz = np.array(output[i][4])   # c
    joint_4_xyz = np.array(output[i][18])   # d

    # the vectors of above three joints.
    vector1 = joint_2_xyz - joint_1_xyz
    vector2 = joint_4_xyz - joint_3_xyz     # Fig_11_2 m09 Active straight raise
    # Vector2 = (1, 0, 0)   # Fig_12_2 m11 Trunk stability

    # the norm of vectors
    norm_vector1 = np.sqrt(vector1.dot(vector1))
    norm_vector2 = np.sqrt(vector2.dot(vector2))
    # the dot product of vectors
    dot_product = vector1.dot(vector2)
    # the radians of the angle
    angle_cos = dot_product / (norm_vector1 * norm_vector2)
    # convert to an angle value
    angle_deg = (np.arccos(angle_cos) * 180) / np.pi

    angleList.append(angle_deg)
#print(angleList)
print('angle Max : ', max(angleList))

angle Max :  84.573393707732


In [4]:
### 최대각도 구하기 #2 ###

max_angleList = []
CMP = 20 #최댓값 구하기 위해 비교할 앞,뒤 각각의 frame 개수

for i in range(CMP,len(angleList)-CMP):
    for j in range(1,CMP+1):
        if (angleList[i] > angleList[i-j]) and (angleList[i] > angleList[i+j]):
            if (j == CMP):
                max_angleList.append(angleList[i])
        else:
            break

print(max_angleList)

[84.573393707732]
